In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Lasso


from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score

from joblib import dump, load

## Lade Daten

In [3]:
df = pd.read_excel("Q4 2022_All Data.xlsx", skiprows=1)
df.fillna(0, inplace=True)
df

,Year,Quater,Country,PC_B2B,PC_B2C,LH_B2B,LH_B2C,Number_Brokers,Number_Salesmen,Number_Tied_Agents,...,Net_Invests_Total,Share_TV,Share_Print,Share_OOH,Share_Radio,Share_Search,Share_Social,Share_Programmatic,Share_of_Advertising,Consideration
0,2022,Q4,Australia,48641,931266.0,0,0,0,0,0,...,5.809023e+06,2.497012e+06,0.000000,256079.921788,1.675773e+05,2.075053e+06,1.083590e+05,0.000000e+00,0.110010,37
1,2022,Q4,Brazil,104273,136764.0,0,0,115172,0,0,...,8.171517e+05,2.259220e+05,4814.137721,160783.668495,8.395308e+04,7.217748e+04,1.396111e+05,0.000000e+00,0.002320,47
2,2022,Q4,France,26361,271226.0,0,50796,0,0,0,...,6.377761e+06,2.917838e+06,440916.666667,0.000000,4.911567e+05,7.991485e+05,5.142997e+05,6.167238e+05,0.072720,26
3,2022,Q4,Germany,83412,334275.0,125270,232911,0,3444,8002,...,1.429130e+07,4.589026e+06,0.000000,3145.512605,1.394159e+06,3.054769e+06,1.092868e+06,3.231758e+06,0.077748,46
4,2022,Q4,Italy,82708,319195.0,459,55462,0,21823,2275,...,1.207884e+06,7.262057e+05,33606.557377,0.000000,0.000000e+00,5.235727e+04,5.669344e+04,3.953025e+04,0.068606,58
5,2022,Q4,Switzerland,10377,32636.0,307,1746,0,1019,0,...,7.736967e+05,1.750125e+04,0.000000,51596.871815,1.369018e+04,2.261660e+05,2.235388e+05,5.711265e+04,0.014363,41


## Datenmanagement

In [4]:
df["pol_total"] = df.PC_B2B + df.PC_B2C + df.LH_B2B + df.LH_B2C
df["lh_anteil"] = (df.LH_B2B + df.LH_B2C) / df.pol_total
df["percent_TV"] = df.Share_TV / df.Net_Invests_Total
df["Switzerland"] = df.Country == "Switzerland"
df_use = df.loc[df.Net_Invests_Total > 0]
df_use

,Year,Quater,Country,PC_B2B,PC_B2C,LH_B2B,LH_B2C,Number_Brokers,Number_Salesmen,Number_Tied_Agents,...,Share_Radio,Share_Search,Share_Social,Share_Programmatic,Share_of_Advertising,Consideration,pol_total,lh_anteil,percent_TV,Switzerland
0,2022,Q4,Australia,48641,931266.0,0,0,0,0,0,...,1.675773e+05,2.075053e+06,1.083590e+05,0.000000e+00,0.110010,37,979907.0,0.000000,0.429851,False
1,2022,Q4,Brazil,104273,136764.0,0,0,115172,0,0,...,8.395308e+04,7.217748e+04,1.396111e+05,0.000000e+00,0.002320,47,241037.0,0.000000,0.276475,False
2,2022,Q4,France,26361,271226.0,0,50796,0,0,0,...,4.911567e+05,7.991485e+05,5.142997e+05,6.167238e+05,0.072720,26,348383.0,0.145805,0.457502,False
3,2022,Q4,Germany,83412,334275.0,125270,232911,0,3444,8002,...,1.394159e+06,3.054769e+06,1.092868e+06,3.231758e+06,0.077748,46,775868.0,0.461652,0.321106,False
4,2022,Q4,Italy,82708,319195.0,459,55462,0,21823,2275,...,0.000000e+00,5.235727e+04,5.669344e+04,3.953025e+04,0.068606,58,457824.0,0.122145,0.601222,False
5,2022,Q4,Switzerland,10377,32636.0,307,1746,0,1019,0,...,1.369018e+04,2.261660e+05,2.235388e+05,5.711265e+04,0.014363,41,45066.0,0.045555,0.022620,True


## Modelle laden

In [5]:
q_modell = load('./model/quaterly_model.joblib')
scaler = load('./model/rescale60to50.joblib')

## Vektoren definieren & Prediciton

In [6]:
use_cols = ['Net_Invests_Total',  "percent_TV", "Number_Tied_Agents", 'lh_anteil', "Switzerland"]
X = df_use[use_cols].values
X

array([[5809023.412671125, 0.42985055604585387, 0, 0.0, False],
       [817151.7336989642, 0.2764749672114871, 0, 0.0, False],
       [6377760.602085325, 0.4575019289409731, 0, 0.14580504789269408,
        False],
       [14291300.183413059, 0.32110629214064634, 8002,
        0.46165198203818175, False],
       [1207883.5893442624, 0.6012216277391116, 2275,
        0.12214519116516391, False],
       [773696.7169051104, 0.022620296162065817, 0, 0.04555540762437314,
        True]], dtype=object)

In [7]:
pred60 = q_modell.predict(X)
pred50 = scaler.predict(pred60.reshape(-1, 1))

In [8]:
out = df_use.copy()
out["policies_60"] = pred60
out["policies_50"] = pred50
out_cols = ['Year', 'Quater', 'Country', 'Net_Invests_Total', 'policies_60', 'policies_50']
out[out_cols].to_excel("PREDICTION_Q4.xlsx", index=False)